In [24]:
from gspread_dataframe import *
from gspread_formatting import *
import imp
import pandas as pd
import pickle as pkl
import re
import numpy as np

In [3]:
t3 = imp.load_source('tab3', 'modules/tab3.py')
s = imp.load_source('patch', 'modules/patcher.py')

In [19]:
dpath = 'config/data/'
trigs = s.get_dict_value('triggers.pkl', 2)
outlay = s.get_dict(dpath + 'outlay3.pkl')

In [9]:
trigger = list(trigs[channel].values())[0]

In [12]:
tmp

array([nan, 'Сегмент', 'Новые подписчики', 'Старые подписчики',
       'Новые клиенты', 'Активные (частые)', 'Активные (остальные)',
       'Предотток (активные)', 'Предотток (новые)', 'Отток (активные)',
       'Отток (новые)', nan, nan, nan], dtype=object)

In [11]:
print(np.where(tmp == trigger)[0])

[]


In [23]:
tmp = gdf.index.values
for channel in trigs.keys():
    trigger = list(trigs[channel].values())[0]
    outlay[channel] = np.where(tmp == trigger)[0][0]
    print(np.where(tmp == trigger)[0])
#put_dictionary(dpath+'outlay3.pkl', outlay)

[31]
[447]
[579]
[607]


In [22]:
outlay

{'email': 31, 'wp': 447, 'seasonal': 523, 'sms': 607}

In [8]:
t3.reindex_outlay(gdf, trigs)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [2]:
from datetime import datetime as dt
from datetime import timedelta
d = imp.load_source('dashboard', 'dashboard.py')
g = imp.load_source('gsync', 'modules/gCanvas.py')
c = imp.load_source('conf', 'config/conf.py')
r = imp.load_source('report', 'modules/reader.py')
s = imp.load_source('patch', 'modules/patcher.py')

In [83]:
from google.cloud import bigquery as bq
from google.oauth2 import service_account

In [95]:
credentials = service_account.Credentials.from_service_account_file('config/creds_bq.json')
creds = credentials.with_scopes(['https://www.googleapis.com/auth/cloud-platform'])

In [96]:
client = bq.Client(project='outofcloud-analytics',
                   credentials=creds)

In [86]:
project='outofcloud-analytics'

In [91]:
name = "analytics." + dt.strftime(dt.today(), format="%Y%m") + '_perekrestok'

In [88]:
import pandas_gbq as pbq

In [101]:
pbq.to_gbq(orders, name, project, if_exists='replace')



0it [00:00, ?it/s]

1it [00:07,  7.78s/it]

GenericGBQException: Reason: 403 Quota exceeded: Your project exceeded quota for free storage for projects. For more information, see https://cloud.google.com/bigquery/troubleshooting-errors

In [58]:
orders = pd.read_feather('data/cohort/orders.f')

In [31]:
orders['date_ord'] = orders.date_ord.apply(lambda x: x.strftime(format="%Y-%m"))

### Monthly

#### 7 tab

In [26]:
def read_clients(clients_file):
    df = pd.read_csv(clients_file, sep=';', low_memory=False)
    df.columns = ['id_cli', 'email', 'id_cust', 'date_reg']
    df = df.loc[(df.email.notna()) & (df.id_cust.notna())]
    df = df.loc[~df.id_cust.apply(lambda x: True if len(re.findall('^\D', str(x))) != 0 else False)]
    df['date_reg'] = pd.to_datetime(df.date_reg, format="%d.%m.%Y %H:%M:%S")
    df['id_cust'] = df.id_cust.astype('int64')
    return df

In [33]:
df = read_clients('data/cohort/clients.csv')
orders = pd.read_feather('data/cohort/orders.f')
res = df.set_index('id_cust').join(orders.set_index('id_cust')).reset_index()
res = res[res.revenue.notna()].reset_index(drop=True)
del df, orders

In [37]:
#TODO convert values to first order
res = res.loc[res.date_reg <= res.date_ord]

In [38]:
res['date_reg'] = res.date_reg.apply(lambda x: x.strftime(format="%Y-%m"))
res['date_ord'] = res.date_ord.apply(lambda x: x.strftime(format="%Y-%m"))

In [39]:
coh = res.groupby(['date_reg', 'date_ord', 'id_cli']).agg({"revenue":np.sum}).reset_index()
coh = coh.loc[coh.date_reg >= "2017"].reset_index(drop=True)

In [41]:
df = coh.groupby(['date_reg', 'date_ord']).agg({"revenue": np.sum}).unstack()
df = df.astype(str)
df.applymap(lambda x: x.replace('.0', '')).to_csv('right_allign.csv')

In [42]:
df = coh.groupby(['date_reg', 'date_ord']).agg({"revenue": np.sum}).unstack(0)
df = df.astype(str)
df.applymap(lambda x: x.replace('.0', '')).to_csv('left_allign.csv')

In [ ]:
#Рассчитываем уникальное количество покупателей, которые совершали покупки за период
l = res.groupby(['date_reg', 'date_ord', 'email'])[['price_in_rub', 'id_ord']].agg({"price_in_rub": np.sum, "id_ord": pd.Series.nunique}).reset_index()
l = l.groupby(['date_reg', 'date_ord'])[['price_in_rub', 'id_ord']].agg({"price_in_rub": np.sum, "id_ord": 'count'}).reset_index()
l = l[l.date_reg <= l.date_ord].set_index(['date_reg', 'date_ord'])['id_ord'].unstack()
l.to_excel('flw_orders.xlsx')

In [ ]:
# Рассчитываем отношение кол-ва уникальных покупателей к их количеству в первом месяце

In [ ]:
# Рассчитываем уникальное количество покупок за период

In [ ]:
# Рассчитываем отношение кол-ва уникальных покупок к их количеству в первом месяце

In [ ]:
# Рассчитываем выручку и считаем LTV (накопленную выручку за весь период) для когорт

In [ ]:
# Рассчитываем отношение выручки в выручке в первый месяц

In [ ]:
# Рассчитываем средний чек

In [ ]:
# Рассчитываем выручку на одного клиента

### 8 tab

In [7]:
from datetime import datetime as dt
from datetime import timedelta
d = imp.load_source('dashboard', 'dashboard.py')
g = imp.load_source('gsync', 'modules/gCanvas.py')
c = imp.load_source('conf', 'config/conf.py')
r = imp.load_source('report', 'modules/reader.py')
s = imp.load_source('patch', 'modules/patcher.py')

In [9]:
# orders = pd.read_csv('data/cohort/orders_all.csv', encoding='cp1251', sep=';', index_col=0)
# orders = orders[['ORDER_DATE','CLIENT_ID', 'ORDER_ID', 'TOTAL']]
# orders.columns = ['date_ord', 'id_cust', 'id_ord', 'revenue']
# orders['date_ord'] = pd.to_datetime(orders.date_ord, format="%d.%m.%Y")
# orders['revenue'] = orders['revenue'].apply(lambda x: str(x).replace(',000','').replace(',040', '').replace('-','')).astype(int)
# orders = orders.reset_index(drop=True)

In [37]:
# r.append_new_orders("data/cohort/orders.f", 'orders_all/orders_17_11-34-51.csv')
# r.append_new_orders("data/cohort/orders.f", 'orders_all/orders_2019-12-24_12-28-50.csv')
# orders = r.append_new_orders("data/cohort/orders.f", 'orders_all/orders_new.csv')

In [3]:
def build_segmentation(clients_file, orders, date):
    """
    Returns strategic segmentation dictionary that counts number of clients on condition
    -> clients_file: mindbox csv with fields: mindbox id, email, external id, registration date
    -> orders: feather dataframe from 'data/cohort/orders.f'
    -> date: datetime
    """
    df = pd.read_csv(clients_file, sep=';', low_memory=False)
    df.columns = ['id_cli', 'email', 'id_cust', 'date_reg']
    df = df.loc[~df.id_cust.apply(lambda x: True if len(re.findall('^\D', str(x))) != 0 else False)]
    df['date_reg'] = pd.to_datetime(df.date_reg, format="%d.%m.%Y %H:%M:%S")
    df['id_cust'] = df.id_cust.astype('int64')
    
    res = df.set_index('id_cust').join(orders.set_index('id_cust')).reset_index()
    dusr = {}

    def get_user_count(df, column,
                       dt_min, dt_max,
                       ord_min=None, ord_max=None):
        res = df.loc[(df[column] <= dt_max) & (df[column] >= dt_min)]
        res = res.groupby(['email']).agg({'id_ord':'count'}).reset_index()
        if column == 'date_reg':
            return res.loc[res.id_ord == 0, 'email'].nunique()
        else:
            if ord_min != None:
                res = res.loc[res.id_ord >= ord_min]
            elif ord_max != None:
                res = res.loc[res.id_ord <= ord_max]
            return res.email.nunique()

    # Новые подписчики не покупали и попали в базу <= 14 дней назад
    dt_min = date - timedelta(days=14)
    dt_max = date

    flws_new = get_user_count(res, 'date_reg', dt_min, dt_max)
    dusr['flws_new'] = flws_new
    del flws_new

    # Старые подписчики не покупали и попали в базу > 14 дней назад
    dt_min = res.date_reg.min()
    dt_max = date - timedelta(days=15)

    flws_old = get_user_count(res, 'date_reg', dt_min, dt_max)
    dusr['flws_old'] = flws_old    
    del flws_old
    
    res = res.loc[res.date_ord.notna()]
    
    # Новые клиенты кол-во покупок от 1 до 3 и последняя <= 7 дней назад
    dt_min = date - timedelta(days=7)
    dt_max = date
    cli_new = get_user_count(res, 'date_ord', dt_min, dt_max, 1, 3)
    dusr['cli_new'] = cli_new    
    del cli_new

    # Предотток (новые) кол-во покупок от 1 до 3 и последняя 8 <= x <= 45 дней назад
    dt_min = date - timedelta(days=45)
    dt_max = date - timedelta(days=8)
    prechurn_new = get_user_count(res, 'date_ord', dt_min, dt_max, 1, 3)
    dusr['prechurn_new'] = prechurn_new
    del prechurn_new

    # Предотток (активные) совершили больше 3 покупок и последняя 30 <= x <= 45 дней назад
    dt_min = date - timedelta(days=45)
    dt_max = date - timedelta(days=30)
    prechurn = get_user_count(res, 'date_ord', dt_min, dt_max, 4)
    dusr['prechurn'] = prechurn
    del prechurn

    # Отток (новые) кол-во покупок от 1 до 3 и последняя > 45 дней назад 
    dt_min = res.date_reg.min()
    dt_max = date - timedelta(days=46)
    churn_new = get_user_count(res, 'date_ord', dt_min, dt_max, 1, 3)
    dusr['churn_new'] = churn_new
    del churn_new

    # Отток (активные) совершили больше 3 покупок и последняя покупка >45 дней назад
    dt_min = res.date_reg.min()
    dt_max = date - timedelta(days=46)
    churn = get_user_count(res, 'date_ord', dt_min, dt_max, 4)
    dusr['churn'] = churn
    del churn

    # Активные (частые) совершили больше 3 покупок и последняя <= 10 дней назад
    dt_min = date - timedelta(days=10)
    dt_max = date
    active = get_user_count(res, 'date_ord', dt_min, dt_max, 4)
    dusr['active'] = active
    del active

    # Активные (остальные) совершили больше 3 покупок и последняя  10 <= x < 30 дней назад
    dt_min = date - timedelta(days=30)
    dt_max = date - timedelta(days=10)
    active_rest = get_user_count(res, 'date_ord', dt_min, dt_max, 4)
    dusr['active_rest'] = active_rest
    del active_rest

    return dusr

In [61]:
#mindbox Клиент -> Действие -> Первое в истории клиента до регистрации
#email, внешний идентификатор, дата регистрации
orders = pd.read_feather('data/cohort/orders.f')

In [63]:
orders.date_ord.max()

Timestamp('2020-01-13 00:00:00')

In [5]:
dates = []
max_date = orders.date_ord.max()
for i in range(15):
    dates.append(max_date)
    max_date -= timedelta(days=7)

In [6]:
segments = [build_segmentation('data/cohort/clients.csv', orders, date) for date in dates]

In [7]:
pd.DataFrame(segments).sort_index(ascending=False).T

,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0
flws_new,7169,7603,7202,6882,6984,6974,8064,11064,12943,10646,8747,9403,12067,11098,8533
flws_old,299902,303475,306995,310931,314434,317832,321333,324752,329184,335355,341975,346452,350899,355702,362298
cli_new,24158,25323,23621,25095,25865,27591,28063,28964,33997,32264,32200,29687,38521,17865,29399
prechurn_new,53129,54890,57105,58716,60455,61635,63468,64983,66579,71407,74393,77164,79408,87060,84000
prechurn,645,632,677,754,735,737,753,713,692,739,780,845,967,1144,1924
churn_new,85328,87993,90836,93653,96741,100339,103630,107297,111241,114419,117968,121645,125593,129859,135050
churn,29914,31088,32254,33496,34694,35976,37251,38615,40071,41421,42858,44376,45933,47442,49211
active,164,196,175,166,163,180,195,209,341,317,619,191,235,172,197
active_rest,1753,1865,1887,1878,1866,1860,1865,2038,2101,2308,2536,4599,4252,4135,1805


In [50]:
orders.groupby(['id_cust']).agg({'id_ord':'count'})

,id_ord
id_cust,
84,16
553,15
9576,13
14299,9
14359,2
...,...
918902,1
918913,1
918959,1


In [31]:
segments

[{'flws_new': 8533,
  'flws_old': 362298,
  'cli_new': 29399,
  'prechurn_new': 84000,
  'prechurn': 1924,
  'churn_new': 135050,
  'churn': 49211,
  'active': 197,
  'active_rest': 1805},
 {'flws_new': 11098,
  'flws_old': 355702,
  'cli_new': 17865,
  'prechurn_new': 87060,
  'prechurn': 1144,
  'churn_new': 129859,
  'churn': 47442,
  'active': 172,
  'active_rest': 4135},
 {'flws_new': 12067,
  'flws_old': 350899,
  'cli_new': 38521,
  'prechurn_new': 79408,
  'prechurn': 967,
  'churn_new': 125593,
  'churn': 45933,
  'active': 235,
  'active_rest': 4252},
 {'flws_new': 9403,
  'flws_old': 346452,
  'cli_new': 29687,
  'prechurn_new': 77164,
  'prechurn': 845,
  'churn_new': 121645,
  'churn': 44376,
  'active': 191,
  'active_rest': 4599},
 {'flws_new': 8747,
  'flws_old': 341975,
  'cli_new': 32200,
  'prechurn_new': 74393,
  'prechurn': 780,
  'churn_new': 117968,
  'churn': 42858,
  'active': 619,
  'active_rest': 2536},
 {'flws_new': 10646,
  'flws_old': 335355,
  'cli_new': 

In [141]:
list(d[0])

[8465, 363252, 29476, 84275, 1932, 135458, 49360, 197, 1811]

In [146]:
d = [list(d.values()) for d in segments]

In [148]:
import itertools

In [151]:
d=list(itertools.chain.from_iterable(d))

In [156]:
import numpy as np

In [167]:
mx = np.array(d).reshape(6,9).T

In [176]:
pd.DataFrame(mx).to_csv('out.csv', sep=',')

In [221]:
res = [8465, 363252, 29476, 84275, 1932, 135458, 49360, 197, 1811]

In [220]:
res = []
maxm = len(list(r[7].values()))
for i in range(maxm):
    res.append(list(r[7].values())[i][0].row)

TypeError: 'int' object is not subscriptable

In [206]:
i = 0
for key, val in r[7].items():
    r[7][key]  = res[i] + 1 
    i += 1 

In [222]:
patch = []
i = 0
for (name, row), val in zip(r[7].items(), res):
    print(name, row, val)
#       patch.append(gspread.models.Cell(row, self.limit,
#                 str(self.df.loc[self.df.group == group_type, x].values[0])))

Новые подписчики 4 8465
Старые подписчики 5 363252
Новые клиенты 6 29476
Активные (частые) 7 84275
Активные (остальные) 8 1932
Предотток (активные) 9 135458
Предотток (новые) 10 49360
Отток (активные) 11 197
Отток (новые) 12 1811


In [104]:
%%time
dusr = build_segmentation('data/cohort/clients.csv', orders, dt.today() - timedelta(days=15))

CPU times: user 12.6 s, sys: 363 ms, total: 13 s
Wall time: 7.41 s


### Weekly

#### tab3

In [3]:
g = imp.load_source('gsync', 'modules/gCanvas.py')
conn = g.Connection("new test dashboard")
name = "8. Страт. сегментация"
ws = conn.get_sheet_by_name(name)
gdf = get_as_dataframe(ws, index_col = 0, evaluate_formulas=True)

In [4]:
gdf.columns = gdf.columns.str.replace('Unnamed: \d+', '')

In [51]:
tmp[2]['email']['(v2) Триггер_4 "Брошенный просмотр категории"'] = "Брошенный просмотр категории (v2)"

In [52]:
tmp[2]['email']['20200207_promocode_cooktest'] = "20200207_promocode_cooktest" 
tmp[2]['email']['20200207_doi1_cooktest'] = "20200207_doi1_cooktest"

In [56]:
tmp[2]['seasonal']['20200121_thanks_cybermonday'] = "Cybermonday Спасибо"
tmp[2]['seasonal']['20200120_doi_cybermonday'] = "Cybermonday DOI"

In [39]:
email = {'Триггер 1. Брошенная корзина': 'Брошенная корзина email',
  'Welcome 1. Приветствие и промокод NEW': 'Welcome 1 new email',
  'Триггер_2_Брошенный просмотр': 'Брошенный просмотр товара email',
  'Триггер_4 "Брошенный просмотр категории"': 'Брошенный просмотр категории email',
  'Триггер_6 "Неделя до ДР"': 'Неделя до ДР email (Триггер 6)',
  'Триггер_6.1 "Неделя до ДР"': 'Неделя до ДР email (Триггер 6.1)',
  'Триггер_5 "Реактивация"': 'Реактивация email',
  'Триггер_7_Товар появился в наличии': 'Товар появился в наличии email',
  'Триггер_3_Напоминание о промокоде A': 'Напоминание о промокоде А email',
  'Триггер_3_Напоминание о промокоде В': 'Напоминание о промокоде В email',
  'Email Отток Активные': 'Отток Активные email',
  'Email trig_water_reminder': 'trig_water_reminder email',
  'Триггер_12_Напоминание о промокоде NEW_2': 'Напоминание о промокоде NEW_2',
  'Триггер_13_Напоминание о промокоде NEW_3': 'Напоминание о промокоде NEW_3',
  'Триггер Додо Пицца': 'Додо Пицца',
  'Триггер Мираторг + Tefal': 'Мираторг + Tefal',
  'DOI Интерактивный тест': 'DOI Интерактивный тест',
  'DOI Интерактивный тест Спасибо': 'DOI Интерактивный тест Спасибо',
  'Пожалуйста, подтвердите подписку (Facebook Leads)': 'Пожалуйста, подтвердите подписку (Facebook Leads)',
  'Ваш промокод уже здесь! (Facebook Leads)': 'Ваш промокод уже здесь! (Facebook Leads)',
  'Рассылка "Предотток с офером"':'Рассылка "Предотток с офером"',
  'Триггер 17. Брошенная корзина с оффером':'Брошенная корзина с оффером',
  'Триггер_14_ "Онлайн дешевле"':'Онлайн дешевле',
  'Письмо с промокодом Babyblog (для старых)':'Письмо с промокодом Babyblog (для старых)',
  'Письмо с промокодом Babyblog (для новых)':'Письмо с промокодом Babyblog (для новых)',
  'DOI BabyBlog':'DOI BabyBlog',
  'Персональный промокод на скидку — в письме! 🤤':'Персональный промокод на скидку — в письме! 🤤',
  'Для вас сюрприз! Откройте письмо 😉':'Для вас сюрприз! Откройте письмо 😉',
  'Вам подарок! 🎁':'Вам подарок!'}

In [41]:
tmp[2]['email'] = email
tmp[2]['seasonal'] = seasonal
tmp[2]['sms'] = {'Копия Ёлка SMS': 'Копия Елка SMS',
            'SMS с опросом':'SMS с опросом',
            'SMS IQOS promocodes':'SMS IQOS promocodes'}

In [22]:
seasonal = {
 'NY landing 2020 DOI':'NY landing 2020 DOI',
 'NY landing 2020 DOI promocode': 'NY landing 2020 DOI promocode',
 'DOI Черная пятница': 'DOI Черная пятница',
 'DOI Черная пятница Спасибо': 'DOI Черная пятница Спасибо',
 'DOI Ёлки': 'DOI Елки',
 'Ёлки Спасибо': 'Елки Спасибо'}

In [26]:
tmp = `s.get_dict('config/data/triggers.pkl')

In [108]:
tmp = pd.read_csv('orders_new.csv', sep = ';')

In [116]:
sum(tmp.id_cust.isin(df.id_cli.unique()))

15478

In [113]:
sum(df.id_cust.isin(tmp.id_cust.unique()))

0

In [117]:
del tmp

In [93]:
sum(df.id_cust.isin(orders.id_cust))

0

In [90]:
df.id_cust.sort_values()

377413         1
552656    111458
388996    122333
507226      1309
539263    140658
           ...  
673402       NaN
673405       NaN
673406       NaN
673407       NaN
673410       NaN
Name: id_cust, Length: 673411, dtype: object

In [25]:
#tmp

In [59]:
z = {'email': 31, 'wp': 486, 'seasonal': 562, 'sms': 674}

In [109]:
tmp.columns

Index(['ID заказа', 'Дата создания', 'Статус', 'Сумма с учетом доставки, руб.',
       'ID Клиента', 'Клиент', 'Телефон клиента', 'E-Mail клиента'],
      dtype='object')

In [110]:
tmp = tmp[['ID заказа', 'Дата создания', 'Сумма с учетом доставки, руб.', 'ID Клиента', 'E-Mail клиента']]

In [111]:
tmp.columns = ['id_ord', 'date_ord', 'revenue', 'id_cust', 'email']

In [43]:
s.put_dict('config/data/triggers.pkl', tmp)

In [16]:
tmp[2] = z

In [15]:
z['sms'] = {'Копия Ёлка SMS': 'Копия Елка SMS',
            'SMS с опросом':'SMS с опросом',
            'SMS IQOS promocodes':'SMS IQOS promocodes'}

In [6]:
z = s.get_dict_value('triggers.pkl', 2)

In [8]:
z['email'] = email

In [14]:
g = imp.load_source('gsync', 'modules/gCanvas.py')
conn = g.Connection("new test dashboard")
name = "3. Триггеры — стат. нед."
ws = conn.get_sheet_by_name(name)
gdf = get_as_dataframe(ws, index_col = 0, evaluate_formulas=True)

In [ ]:
g.gCanvas(self.connector.get_sheet_by_name(sheet))

In [13]:
df = r.get_report('report.xlsx', 0)
df = df[df.channel != 'Ручные рассылки']

In [26]:
z = {'email': 31, 'wp': 447, 'seasonal': 523, 'sms': 607}

In [25]:
s.get_dict('config/data/outlay3.pkl')

{'email': 31, 'wp': 451, 'seasonal': 583, 'sms': 611}

In [60]:
s.put_dict('config/data/outlay3.pkl', z)

In [ ]:
# row 83 

In [23]:
conn = g.Connection("new test dashboard")
name = "3. Триггеры — стат. нед."
z = g.gCanvas(conn.get_sheet_by_name(name))
patcher = s.Patcher(df, 2, z.max_rows, gdf)

Определите категорию кампании для 3 новых:

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> (v2) Триггер_4 "Брошенный просмотр категории": 
Брошенный просмотр категории (v2)
Повторите ввод в согласии с инструкцией

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> (v2) Триггер_4 "Брошенный просмотр категории": 

Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets
Брошенный просмотр категории (v2)
Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> 20200121_thanks_cybermonday: 
2
Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets
Киберпонедельник Спасибо
Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> 20200120_doi_cybermonday: 
Киберпонедельник DOI
Повторите ввод в согласии с инструкцией

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для

In [8]:
patcher = s.Patcher(df, 2, 24, gdf)

Определите категорию кампании для 9 новых:

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> Триггер 17. Брошенная корзина с оффером: 

Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets
Брошенная корзина с оффером
Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> NY landing 2020 DOI: 
2
Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> NY landing 2020 DOI promocode: 
2
Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> Триггер_14_ "Онлайн дешевле": 

Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets
Онлай

In [24]:
patcher.gdf

,,Описание триггера,Условия отправки,Ссылка на визуал,Unnamed: 4,август,Unnamed: 6,Unnamed: 7,Unnamed: 8,сентябрь,...,Unnamed: 24,Unnamed: 25,Январь,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Февраль,Unnamed: 32,Unnamed: 33
0,номер недели,,,,,32,33,34,35,36,...,51,52,1,2,3,4,5,6,7,8
1,даты,,,,,01-14.08,15-21.08,22-28.08,29.08-04.09,05-11.09,...,17-23.12,24-30.12,31.12-06.01,07.01-13.01,14.01-20.01,21.01-27.01,28.01-03.02,04.02-10.02,11.02-17.02,18.02-24.02
2,Трафик на сайте,,,,GA,711568,392088,379103,405548,412654,...,742072,900413,483802,484135,516790,,,,,
3,Регистраций,,,,,,,,,,...,,,,,,,,,,
4,Email,,,,,,,,,,...,9397,27231,8181,8932,11923,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,SMS с опросом,,"Потвор ""Онлайн дешевле""",https://perekrestok.mindbox.ru/campaigns/14/ma...,,,,,,,...,,,,,,,,,,
656,Доставлено,,,,Mb,,,,,,...,2596,2690,1284,2345,3627,,,,,
657,SMS IQOS promocodes,,"Потвор ""Онлайн дешевле""",https://perekrestok.mindbox.ru/campaigns/14/ma...,,,,,,,...,,,,,,,,,,
658,Доставлено,,,,Mb,,,,,,...,,,,,1,,,,,


#### tab5

In [117]:
df = r.get_report('report.xlsx', 0)
df = df[df.channel == 'Ручные рассылки']

In [118]:
s.Patcher(df, 4, 123).patch

[<Cell R123C1 '21.01'>,
 <Cell R123C2 '18:31'>,
 <Cell R123C3 '20200121_cybermonday_warming_up_210120'>,
 <Cell R123C4 'Киберпонедельник'>,
 <Cell R123C5 '@{ if IsEmpty(Recipient.OnlyStandardFirstName) }Такое точно нельзя пропустить! @{ else } ${ Recipient.OnlyStandardFirstName }, такое точно нельзя пропустить! @{end if}'>,
 <Cell R123C6 'https://email-services.mindbox.ru/v1/projects/1309/email/previewtemplate/?uniqueKey=4539628431335227392&hash=1033C6CFC43FD62160789B72D18B9B40A05770D5&format=Html'>,
 <Cell R123C7 'Email'>,
 <Cell R123C8 '238826'>,
 <Cell R123C9 '50197'>,
 <Cell R123C10 '0,210182308458878%'>,
 <Cell R123C11 '11412'>,
 <Cell R123C12 '0,04778374213862812%'>,
 <Cell R123C13 '0,22734426360141044%'>,
 <Cell R123C14 '311'>,
 <Cell R123C15 '0,0013022032777000828%'>,
 <Cell R123C16 '3498750'>,
 <Cell R123C17 '935'>,
 <Cell R123C18 '3741'>,
 <Cell R123C19 '0,003914984130706037%'>,
 <Cell R124C1 '23.01'>,
 <Cell R124C2 '13:00'>,
 <Cell R124C3 '20200123_usa_У нас тут американский

#### tab4

In [67]:
conn = g.Connection("new test dashboard")
name = "4. Триггеры — инкр. месяц"
z = g.gCanvas(conn.get_sheet_by_name(name))
gdf = z.get_as_df()

In [68]:
df = r.get_report('report_all.xlsx', 0)

In [69]:
months_ru = {'01':'Январь',
             '02':'Февраль',
             '03':'Март',
             '04':'Апрель',
             '05':'Май',
             '06':'Июнь',
             '07':'Июль',
             '08':'Август',
             '09':'Сентябрь',
             '10':'Октябрь',
             '11':'Ноябрь',
             '12':'Декабрь',}

In [70]:
df['date'] = pd.to_datetime(df.date, format="Y-%m-%d %H:%M:%S")\
                            .apply(lambda x: dt.strftime(x, "%m"))
df = df.loc[df.date == '08']
df['date'] = df.date.map(months_ru)
df = df.set_index('date')[['campaign', 'name', 'delivered', 
                           'opened', 'open_rate', 'clicked', 
                           'click_rate', 'CTR', 'unfollowed', 
                           'unfollow_rate', 'revenue', 'orders',
                           'avg_bill', 'order_conversion']]
def get_channel(string):
        string = string.lower()
        if 'sms' in string:
            return 'SMS'
        elif 'web-push' in string or 'wp' in string:
            return 'Web-push'
        else:
            return 'Email'
df['channel'] = df.name.map(get_channel)
orders = pd.read_feather('data/cohort/orders.f')
orders = orders.loc[(orders.date_ord >= "2019-08") & (orders.date_ord < "2019-09")]
orders = r.get_order_info(orders, agg_column='group')

In [71]:
orders

,group,revenue,id_cli,id_ord
0,control,11902930,1674,3278
1,test,221499907,32112,60068


In [79]:
tmp = s.get_dict('data/cell_dicts.pkl')

In [81]:
tmp[1]['test']['Количество участников ЭГ'] = 19
tmp[1]['control']['Количество участников КГ'] = 32

In [83]:
s.put_dict('data/cell_dicts.pkl', tmp)

In [90]:
trigs = s.get_dict('config/data/triggers.pkl')

In [92]:
trigs[1]

{'Число ПОКУПАТЕЛЕЙ в ЭГ за период': 'id_cli',
 'Число ПОКУПОК в ЭГ за период': 'id_ord',
 'Доход за неделю ЭГ': 'revenue',
 'Число ПОКУПАТЕЛЕЙ в КГ за период': 'id_cli',
 'Число ПОКУПОК в  КГ за период': 'id_ord',
 'Доход за неделю КГ': 'revenue'}

In [98]:
gdf.loc['Август'].columns[14:]

Index(['Количество участников ЭГ', 'Число ПОКУПАТЕЛЕЙ в ЭГ за период',
       'Число ПОКУПОК в ЭГ за период', 'Доход по покупкам в ЭГ',
       'Средний чек в ЭГ', 'Конверсия в покупку ', 'Доход на участника ЭГ',
       'Количество участников КГ', 'Число ПОКУПАТЕЛЕЙ в КГ за период',
       'Число ПОКУПОК в КГ за период', 'Доход по покупкам в КГ',
       'Средний чек в КГ', 'Конверсия в покупку в КГ', 'Доход на участника КГ',
       'Чистая конверсия ', 'Чистый прирост заказов', 'Чистый прирост выручки',
       'Чистый прирост выручки, %', 'Значимость'],
      dtype='object')

In [164]:
patch = []
for group_type, info in d.items():
    for name, row in info.items():
        x = t.get(name)
        print(df.loc[df.group == group_type, x].values[0])

#### 2 tab

In [40]:
res="""Инкрементальная выручка (накопительным итогом)
Инкрементальный прирост (накопительным итогом)
Инкрементал общий (недельный)
Инкрементал от триггеров (недельный)
Чистый прирост заказов
Чистый прирост выручки на человека
Чистый прирост выручки на человека, %
Значимость"""

In [42]:
res = res.split("\n")

In [45]:
res

['Инкрементальная выручка (накопительным итогом)',
 'Инкрементальный прирост (накопительным итогом)',
 'Инкрементал общий (недельный)',
 'Инкрементал от триггеров (недельный)',
 'Чистый прирост заказов',
 'Чистый прирост выручки на человека',
 'Чистый прирост выручки на человека, %',
 'Значимость']

In [54]:
triggers = s.get_dict("config/data/attribution.pkl")

In [63]:
new_t = dict(zip(res, np.arange(5, 13)))

In [103]:
d = s.get_dict('data/cell_dicts.pkl')
d = d[1]
t = s.get_dict('config/data/attribution.pkl')
from gspread.utils import rowcol_to_a1 as c2a

In [111]:
d

{'test': {'Число ПОКУПАТЕЛЕЙ в ЭГ за период': 21,
  'Число ПОКУПОК в ЭГ за период': 25,
  'Доход за неделю ЭГ': 26,
  'Количество участников ЭГ': 19},
 'control': {'Число ПОКУПАТЕЛЕЙ в КГ за период': 34,
  'Число ПОКУПОК в  КГ за период': 38,
  'Доход за неделю КГ': 39,
  'Количество участников КГ': 32}}

In [ ]:
'Прирост участников ЭГ': 20
'Прирост покупателей в ЭГ': 22
'',
'',


In [ ]:
c2a()

In [ ]:
t[1] = {
    5:''
}

In [ ]:
Описание алгоритма расчёта для инкрементала:
    Инкрементальная выручка (накопительным итогом) == Чистый прирост выручки на человека, умноженное на кол-во участников ЭГ
    Инкрементальный прирост (накопительным итогом) == Инкрементальная выручка (накопительным итогом) разделить на Накопленный доход по покупкам для обеих групп
    Инкрементал общий (недельный) == Доход за неделю на участника ЭГ делим на Доход за неделю участника КГ и умножаем на кол-во участников ЭГ
    Инкрементал от триггеров (недельный) == ?
    Чистый прирост заказов == Кол-во заказов за текущий период - Кол-во заказов за прошлый
    Чистый прирост выручки на человека == Доход на участника ЭГ / Доход на участника КГ
    Чистый прирост выручки на человека, % == Чистый прирост выручки на человека в процентах
    Значимость == Чистый прирост выручки на человека разделить на сумму дохода обеих групп